In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/OCR/trainset/trainset.zip -d /content

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/train/TRAIN_06671.png  
  inflating: /content/__MACOSX/train/._TRAIN_06671.png  
  inflating: /content/train/TRAIN_38947.png  
  inflating: /content/__MACOSX/train/._TRAIN_38947.png  
  inflating: /content/train/TRAIN_40624.png  
  inflating: /content/__MACOSX/train/._TRAIN_40624.png  
  inflating: /content/train/TRAIN_09542.png  
  inflating: /content/__MACOSX/train/._TRAIN_09542.png  
  inflating: /content/train/TRAIN_57358.png  
  inflating: /content/__MACOSX/train/._TRAIN_57358.png  
  inflating: /content/train/TRAIN_05350.png  
  inflating: /content/__MACOSX/train/._TRAIN_05350.png  
  inflating: /content/train/TRAIN_43305.png  
  inflating: /content/__MACOSX/train/._TRAIN_43305.png  
  inflating: /content/train/TRAIN_54679.png  
  inflating: /content/__MACOSX/train/._TRAIN_54679.png  
  inflating: /content/train/TRAIN_25080.png  
  inflating: /content/__MACOSX/train/._TRAIN_25080.png  
  inflating: /content/train/TRAIN_531

In [6]:
import pandas as pd
df = pd.read_csv("/content/train.csv")
df

,id,img_path,label
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색
1,TRAIN_00001,./train/TRAIN_00001.png,머
2,TRAIN_00002,./train/TRAIN_00002.png,차차
3,TRAIN_00003,./train/TRAIN_00003.png,써
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다
...,...,...,...
76883,TRAIN_76883,./train/TRAIN_76883.png,회
76884,TRAIN_76884,./train/TRAIN_76884.png,겪다
76885,TRAIN_76885,./train/TRAIN_76885.png,벨트
76886,TRAIN_76886,./train/TRAIN_76886.png,톼


In [7]:
!pip install straug
!apt install imagemagick

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.9 MB/s 
     |████████████████████████████████| 142 kB 48.0 MB/s 
  Created wheel for magickwand: filename=magickwand-0.2-py3-none-any.whl size=32460 sha256=d30e19dec9f88fe40f7a7009d71fc32c966be678d2fdaaef31346fa1b821c4df
  Stored in directory: /root/.cache/pip/wheels/e0/67/16/03c4b8517d22de2c2db6e24b06255017edf2b7e78ab7582778
Successfully built magickwand
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2d

In [1]:
# 에러시 런타임 -> 런타임 다시 시작 후 여기부터 다시 실행
from straug.geometry import Rotate
from straug.pattern import Grid
from straug.blur import DefocusBlur
from straug.camera import JpegCompression, Pixelate

import cv2
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [2]:
df = pd.read_csv("/content/train.csv")

In [4]:
for idx in range(len(df)):
  img = Image.open(f'{df["img_path"][idx]}')
  label = df['label'][idx]
  aug1 = DefocusBlur()(img, mag=-1, prob=1.0)
  aug2 = JpegCompression()(img, mag=-1, prob=1.0)
  aug3 = Pixelate()(img, mag=-1, prob=1.0)
  aug1.save(f'./train/{df["id"][idx]}_1.png', 'png')
  aug2.save(f'./train/{df["id"][idx]}_2.png', 'png')
  aug3.save(f'./train/{df["id"][idx]}_3.png', 'png')

  aug_df = pd.DataFrame({
      'id' : [f'{df["id"][idx]}_1', f'{df["id"][idx]}_2', f'{df["id"][idx]}_3'],
      'img_path': [f'./train/{df["id"][idx]}_1.png', f'./train/{df["id"][idx]}_2.png', f'./train/{df["id"][idx]}_3.png'],
      'label' : [label, label, label]
  })
  df = pd.concat([df, aug_df])
  df = df.reset_index(drop=True)
df

,id,img_path,label
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색
1,TRAIN_00001,./train/TRAIN_00001.png,머
2,TRAIN_00002,./train/TRAIN_00002.png,차차
3,TRAIN_00003,./train/TRAIN_00003.png,써
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다
...,...,...,...
307547,TRAIN_76886_2,./train/TRAIN_76886_2.png,톼
307548,TRAIN_76886_3,./train/TRAIN_76886_3.png,톼
307549,TRAIN_76887_1,./train/TRAIN_76887_1.png,자리
307550,TRAIN_76887_2,./train/TRAIN_76887_2.png,자리


In [5]:
df.to_csv('/content/train_aug.csv', index=False)